<a href="https://colab.research.google.com/github/kapasi1234/solar-forecasting-using-INSAT-3D-satelite-images/blob/main/GHI_Integrated_Code_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Steps to execute
  1. Order the hdf5 data from MOSDAC website. we have to order two folders one for calculating clear and cloudy pixel values and the other for calculating GHI values.
  2. Mount the drive.
  3. Provide the necessary details and run the "Auto-Download the data" cell.
  4. Download the shape file of your area of interest and place it in the main directory.
  5. Download one tif file from mosdac website and place it in the main directory.
  6. place one sample file for AOD in the main directory.
  7. Mention the AOI and Process and run the "Initial Processing" cell.
  8. Initially, select the process as cloud detection,run the "initialization" cell and run the "cloud detection process" cell to calculate the clear and cloudy pixel for the particular Area
  9. The use the clear and cloudy pixel values to calculate the Cloud Index.
  10. After calculating cloud index, select the process as Forecasting,run the "initialization" cell. Inside initialization cell we need to provide the forecasting algorithm to be used.
  11. we need to download the aerosol data file and perciptable water data file based on the test_data and run the "GHI Calculation process" cell.
  12. In order the evaluate the Model performance, Get the ground data GHI values for that area and correlate with the normal and the forecasted GHI values and also check the error using Root Mean Squared Error metrics. To do that provide the necessary details in the "Error metrics" cell and run it.


In [ ]:
#Mount the drive
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

In [ ]:
#Auto-Download the data
#@title Auto-Download the data { display-mode: "form" }
Username = "" #@param {type:"string"}
Password = "" #@param {type:"string"}
Folder_name_for_data = "" #@param {type:"string"}
Folder_name_for_test_data = "" #@param {type:"string"}
cnopts=pysftp.CnOpts()
cnopts.hostkeys=None
os.makedirs('/content/drive/MyDrive/Project/Data')
os.mkdir('/content/drive/MyDrive/Project/Test_Data')
with pysftp.Connection('103.99.192.65', username=Username, password=Password,cnopts=cnopts,port=22) as sftp:
  # initially, we need to place the order in the mosdac website and the folder name. Note the folder name and pass it as parameter.
  #one folder for calculating clear and cloudy values and other for calculating the GHI values
  # In the get_d method the first option is remote dir path and the second option is local dir path.
  sftp.get_d('Order/'+Folder_name_for_data,'/content/drive/MyDrive/Project/Data')
  sftp.get_d('Order/'+Folder_name_for_test_data,'/content/drive/MyDrive/Project/Test_Data')
  sftp.close()

In [ ]:
##Installing necessary packages
!pip install pysftp
!pip install gdal
!pip install fiona
!pip install osgeo
!pip install shapely
!pip install rasterio
!pip install geopandas
!pip install rioxarray
!pip install xarray
!pip install earthpy
!pip install pvlib
!pip install blockmatching

##-------------------------------------------------------------------------------------------------------------------------##

##Importing the packages
import os
import shutil
import h5py
import glob
import rasterio as rio
import rasterio.plot
import numpy as np
import pandas as pd
from tabulate import tabulate
import matplotlib.pyplot as plt
from PIL import Image
import cv2 as cv
from numpy import savetxt
import pysftp
import math
import datetime
from PIL import Image
import geopandas as gd
import rioxarray as rxr
import xarray as xr
from shapely.geometry import mapping, box
import earthpy as et
import earthpy.plot as ep
import base64
import numpy.ma as ma
import pvlib
from pvlib import *
from pvlib.location import Location
from blockmatching import *
from datetime import timedelta
from types import LambdaType
from sklearn.metrics import mean_squared_error
from math import sqrt

##-------------------------------------------------------------------------------------------------------------------------##


#initializations
#@title Initial Processing { display-mode: "form" }
AOI = "pokhran" #@param ["pokhran", "tirupati", "kutch", "trichy"]
Process = "Cloud Detection" #@param ["Cloud Detection", "Forecasting"]


##-------------------------------------------------------------------------------------------------------------------------##

# Initial processing of data for cloud mask
def initial_processing_cloud_mask(filepath,filename,dir,AOI):
  f=h5py.File(filepath,"r")
  vis=np.squeeze(np.array(f['IMG_VIS'][:]))
  sun_elevation=np.array(f['Sun_Elevation'][:])
  sun_elevation=np.squeeze(sun_elevation)/100
  solar_zenith=np.sin(np.radians(sun_elevation))
  vis_corrected=vis/solar_zenith
  vis_rad=np.uint8(0.112135*vis_corrected-3.30421)
  for x in np.nditer(vis_rad,op_flags=["readwrite"]):
    x[x<0]=0
  sample_file=rio.open(dir+"/sample_file.tif")
  desired_band=[0]
  with rio.open('/content/tiff_files/vis/'+filename.split('_')[1]+"_"+filename.split('_')[2]+"_vis.tiff","w",driver="Gtiff",height=sample_file.height,width=sample_file.width,count=1,dtype=vis_rad.dtype,crs=sample_file.crs,transform=sample_file.transform) as dst:
    dst.write(vis_rad,indexes=1)
  vis_t=rxr.open_rasterio('/content/tiff_files/vis/'+filename.split('_')[1]+"_"+filename.split('_')[2]+"_vis.tiff",variable=desired_band).squeeze()
  Area_of_Interest=gd.read_file(dir+"/Shape_file/"+AOI+"/"+AOI+".shp")
  Area_of_Interest.geometry
  crop_bound_box = [box(*Area_of_Interest.total_bounds)]
  crop_bound_box[0]
  Area_of_Interest.geometry[0]
  vis_clipped = vis_t.rio.clip(crop_bound_box,crs=Area_of_Interest.crs,all_touched=False,from_disk=False).squeeze()
  plt.imsave('/content/vis_img_'+AOI+'/'+filename.split('_')[1]+'_'+filename.split('_')[2]+"_vis.png",vis_rad)
  return vis_clipped

# calculating the cloud mask values based on the visible radiance data
def cloud_mask(filepath,dir,imgpath,AOI):
  f=h5py.File(filepath,"r")
  filename=f.split('/')[8]
  img=cv.imread(imgpath)
  Z = img.reshape((-1,3))
  Z = np.float32(Z)
  criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 10, 1.0)
  K = 2
  ret,label,center=cv.kmeans(Z,K,None,criteria,10,cv.ADAPTIVE_THRESH_MEAN_C)
  center = np.uint8(center)
  res = center[label.flatten()]
  res2 = res.reshape((img.shape))
  res2bw = cv.cvtColor(res2, cv.COLOR_BGR2GRAY)
  colors = np.unique(res2bw)
  colors = colors[1:]  
  for i in range(0, len(colors)):
    filter_mask = (res2bw == colors[i]).astype(int)
  filter_mask=filter_mask.astype("int32")
  sample_file=rio.open(dir+"/sample_file.tif")
  desired_band=[0]
  with rio.open("/content/tiff_files/filter_mask/"+filename.split('_')[1]+"_"+filename.split('_')[2]+"_filter_mask.tiff","w",driver="Gtiff",height=file.height,width=file.width,count=1,dtype=filter_mask.dtype,crs=file.crs,transform=file.transform) as dst:
    dst.write(filter_mask,indexes=1)
  filter_mask_tiff=rxr.open_rasterio("/content/tiff_files/filter_mask/"+filename.split('_')[1]+"_"+filename.split('_')[2]+"_filter_mask.tiff",variable=desired_bands).squeeze()
  Area_of_Interest=gd.read_file(dir+"/Shape_file/"+AOI+"/"+AOI+".shp")
  Area_of_Interest.geometry
  crop_bound_box = [box(*Area_of_Interest.total_bounds)]
  crop_bound_box[0]
  Area_of_Interest.geometry[0]
  filter_mask_clip=filter_mask_tiff.rio.clip(crop_bound_box,crs=Area_of_Interest.crs,all_touched=False,from_disk=False).squeeze()  
  filter_mask_clip=np.array(filter_mask_clip)
  return filter_mask_clip

##-------------------------------------------------------------------------------------------------------------------------##
#To calculate the GHI
def GHI_Calculation(filepath,filename,dir,AOI,dfname,ci):
  f1=h5py.File(filepath)
  starttime=f1.attrs['Acquisition_Start_Time'].decode('UTF-8')
  endtime=f1.attrs['Acquisition_End_Time'].decode('UTF-8')
  if AOI=='kutch':
    a=0.04
    b=0.96
    altitude=74
    lat=23.334
    lon=70.637
  elif AOI=='pokhran':
    a=0.04
    b=0.96
    altitude=293
    lat=26.916
    lon=71.928
  elif AOI=='tirupati':
    a=0.03
    b=0.97
    altitude=194
    lat=13.627
    lon=79.397
  elif AOI=='trichy':
    a=0.03
    b=0.97
    altitude=87
    lat=10.76
    lon=78.813
  k=a+b*(1-ci)
  k=np.array(k)
  location=Location(lat,lon,altitude=altitude,name=AOI)
  AOD=AOD_calculation(filename,AOI,dir)
  pwd=pwd_calculation(filename,AOI,dir,starttime)
  clearsky=location.get_clearsky(pd.date_range(start=starttime, end=endtime,freq='1min'),model='simplified_solis',aod700=AOD,precipitable_water=pwd)
  clear_ghi=np.array(clearsky["ghi"])
  clear_ghi_mean=np.array(clear_ghi.mean())
  actual_ghi=clear_ghi_mean*k
  date=pd.to_datetime(starttime)
  date1=date+timedelta(hours=5,minutes=30)
  dfname.loc[-1]=[date1,actual_ghi]
  dfname.index=dfname.index+1
  return dfname

#To calculate Aerosol Optical Depth Value for solis method
def AOD_calculation(filename,AOI,dir):
  date=pd.to_datetime(filename.split('_')[1])
  date_time=date.strftime("%Y%m%d")
  AOD_shp=gd.read_file(dir+"/Shape_file/"+AOI+"_AOD/"+AOI+"_AOD.shp")
  sample_file=rio.open(dir+"/sample_AOD_file.nc4")
  desired_bands="AODANA"
  aod4=rxr.open_rasterio(dir+"/sample_AOD_file.nc4",masked=True,variable=desired_bands)
  aod4.rio.set_crs("epsg:4326")
  aod_crs=aod4.rio.crs
  crs=aod_crs
  for subdir1 in os.listdir(dir):
    if subdir1=="aerosol data":
      subdir1=os.path.join(dir,subdir1)
      for subdir2 in os.listdir(subdir1): ## separate folders for each month
        subdir2=os.path.join(subdir1,subdir2)
        for filename1 in os.listdir(subdir2):
          aod_file=os.path.join(subdir2,filename1)
          if filename1.endswith(".nc4"):
            aod_filename=filename1.split('.')[2]
            if aod_filename==date_time:
              aod=h5py.File(aod_file)
              aod=aod.get("AODANA")
              for i in range(0,len(aod)):
                if i==1 or i==2 or i==3:
                  list=aod[i].shape
                  with rio.open("/content/tiff_files/aod/aod_"+filename.split('_')[1]+'_'+filename.split('_')[2]+".tiff","w",driver="Gtiff",height=list[0],width=list[1],count=1,dtype=aod.dtype,crs=crs,transform=sample_file.transform) as dst:
                    dst.write(aod[i],indexes=1)
                  aod_clip=rxr.open_rasterio("/content/tiff_files/aod/aod_"+filename.split('_')[1]+'_'+filename.split('_')[2]+".tiff",variable="AODANA",masked=True).squeeze()
                  crop_bound_box = [box(*AOD_shp.total_bounds)]
                  aod_clipped= aod_clip.rio.clip(crop_bound_box,crs=AOD_shp.crs,all_touched=False,from_disk=False).squeeze() 
                  stacked=np.array(aod_clipped).squeeze()
                  aod_total=np.concatenate(stacked)
  AOD=aod_total.mean()   
  return AOD

#To calculate the pwd value for solis method
def pwd_calculation(filename,AOI,dir,starttime):
  pwd_dir='pwd_'+starttime.split('-')[1]+' '+starttime.split('-')[2].split('T')[0]
  AOD_shp=gd.read_file(dir+"/Shape_file/"+AOI+"/"+AOI+".shp")
  for subdir1 in os.listdir(dir):
    if subdir1==pwd_dir:
      subdir1=os.path.join(dir,subdir1)
      for filename2 in os.listdir(subdir1):
        if filename2.endswith(".nc4"):
          pwd_file=os.path.join(subdir1,filename2)
          sample_file=rio.open(pwd_file)
          desired_bands="TQV"
          pwd=rxr.open_rasterio(pwd_file,masked=True,variable=desired_bands)
          pwd.rio.set_crs("epsg:4326")
          pwd_crs=pwd.rio.crs
          crs=pwd_crs
          f1=h5py.File(pwd_file)
          TQV=np.array(f1.get("TQV"))
          with rio.open("/content/tiff_files/pwd/pwd_"+filename.split('_')[1]+'_'+filename.split('_')[2]+".tiff","w",driver="Gtiff",height=sample_file.height,width=sample_file.width,count=1,dtype=TQV.dtype,crs=crs,transform=sample_file.transform) as dst:
            dst.write(TQV)
          pwd_clip=rxr.open_rasterio("/content/tiff_files/pwd/pwd_"+filename.split('_')[1]+'_'+filename.split('_')[2]+".tiff",variable="AODANA",masked=True).squeeze()
          crop_bound_box = [box(*AOD_shp.total_bounds)]
          pwd_clipped= pwd_clip.rio.clip(crop_bound_box,crs=AOD_shp.crs,all_touched=False,from_disk=False).squeeze() 
          pwd_mean=np.array(pwd_clipped).squeeze().mean()
  return pwd_mean

##-------------------------------------------------------------------------------------------------------------------------##

# Initial processing of data for ghi calculation
def initial_processing(filepath,filename,dir,AOI):
  f=h5py.File(filepath,"r")
  vis=np.squeeze(np.array(f['IMG_VIS'][:]))
  sun_elevation=np.array(f['Sun_Elevation'][:])
  sun_elevation=np.squeeze(sun_elevation)/100
  solar_zenith=np.sin(np.radians(sun_elevation))
  vis_corrected=vis/solar_zenith
  vis_rad=np.uint8(0.112135*vis_corrected-3.30421)
  for x in np.nditer(vis_rad,op_flags=["readwrite"]):
    x[x<0]=0
  sample_file=rio.open(dir+"/sample_file.tif")
  desired_band=[0]
  with rio.open('/content/tiff_files/vis/'+filename.split('_')[1]+"_"+filename.split('_')[2]+"_vis.tiff","w",driver="Gtiff",height=sample_file.height,width=sample_file.width,count=1,dtype=vis_rad.dtype,crs=sample_file.crs,transform=sample_file.transform) as dst:
    dst.write(vis_rad,indexes=1)
  vis_t=rxr.open_rasterio('/content/tiff_files/vis/'+filename.split('_')[1]+"_"+filename.split('_')[2]+"_vis.tiff",variable=desired_band).squeeze()
  return vis_t

# clip the visible data info of area of interest from given file
def clip_AOI(vis,f,AOI,method):
  Area_of_Interest=gd.read_file(dir+"/Shape_file/"+AOI+"/"+AOI+".shp")
  Area_of_Interest.geometry
  crop_bound_box = [box(*Area_of_Interest.total_bounds)]
  crop_bound_box[0]
  Area_of_Interest.geometry[0]
  vis_clipped = vis.rio.clip(crop_bound_box,crs=Area_of_Interest.crs,all_touched=False,from_disk=False).squeeze()
  return vis_clipped
##-------------------------------------------------------------------------------------------------------------------------##
#Forecasting Methods

#Block Matching Method to forecast the GHI
def BM(frame,old_frame):
  background = BackgroundSubtractor(0.01, old_frame.astype(np.uint8), sgm = 1)
  foreground = background.foreground(old_frame.astype(np.uint8))
  frame0 = foreground.copy()
  frame1 = background.foreground(frame.astype(np.uint8))
  width = 4
  height = 4
  XP, YP, XD, YD = block_matching(frame0, frame1, width, height)
  U, V, object_tops, meand = clustering(XD, YD, XP, YP, smooth=10, maxsizegraph=10)
  lars = layers(frame, object_tops, width, height, sigma=10)
  sigma = 10
  r = zeros_like(frame)
  for i in range(len(meand)):
    c = 0.4
    dsy, dsx = int(c * meand[i][0]), int(c * meand[i][1])
    fill_width = (0, abs(dsx)) if dsx < 0 \
    else (abs(dsx), 0)
    fill_height = (0, abs(dsy)) if dsy < 0 \
    else (abs(dsy), 0)
    tmp = pad(lars[i], (fill_height, fill_width), mode='constant')
    if dsy == 0 and dsx < 0:
      tmp = tmp[:, abs(dsx):]
    elif dsy == 0 and dsx > 0:
      tmp = tmp[:, :-abs(dsx)]
    elif dsy > 0 and dsx == 0:
      tmp = tmp[:-abs(dsy), :]
    elif dsy < 0 and dsx == 0:
      tmp = tmp[abs(dsy):, :]
    elif dsy < 0 and dsx < 0:
      tmp = tmp[abs(dsy):, abs(dsx):]
    elif dsy < 0 and dsx > 0:
      tmp = tmp[abs(dsy):, :-abs(dsx)]
    elif dsy > 0 and dsx < 0:
      tmp = tmp[:-abs(dsy), abs(dsx):]
    elif dsy > 0 and dsx > 0:
      tmp = tmp[:-abs(dsy), :-abs(dsx)]
    r = r + tmp
    kernel = ones((sigma, sigma), float32) / sigma ** 2
    forecast = cv.filter2D(r, -1, kernel)
    t1 = forecast.copy()
    t2 = background.background.copy()
    ret, mask = cv.threshold(t1, 5, 255, cv.THRESH_BINARY+cv.THRESH_OTSU)
    mask_inv = cv.bitwise_not(mask)
    img1_bg = cv.bitwise_and(t1,t1,mask = mask)
    img2_fg = cv.bitwise_and(t2,t2,mask = mask_inv)
    dst = cv.add(img1_bg,img2_fg)
    return dst

#Optical Flow Method to forecast the GHI
def optical_flow_Method(ci1,ci2):
  optical_flow = cv.optflow.DualTVL1OpticalFlow_create(0.1,0.1,0.3,3,3,0.01,10,2,0.5,0.1)
  flow=optical_flow.calc(ci1,ci2,None)
  h, w = flow.shape[:2]
  flow=flow
  flow[:,:,0] += np.arange(w)
  flow[:,:,1] += np.arange(h)[:,np.newaxis]
  forecast = cv.remap(ci2, flow, None, cv.INTER_LINEAR)
  forecast_ci=forecast.mean()
  return forecast_ci


## Initializations

In [ ]:
#@title Main Code Execution{ display-mode: "code" }
dir='/content/drive/MyDrive/Project' ## please change the directory name

# to remove the folder 
!rm -r '/content/tiff_files/'
!rm -r '/content/vis_img_'+AOI+'/'

if Process == "Cloud Detection":
  os.makedirs('/content/tiff_files/vis/')
  os.makedirs('/content/tiff_files/filter_mask/')
  os.makedirs('/content/vis_img_'+AOI+'/')
  os.mkdir(dir+'/clear and cloudy pixels/')
  df_clear=pd.DataFrame(columns=['clear_pixels'])
  df_cloudy=pd.DataFrame(columns=['cloudy_pixels'])
elif Process == "Forecasting":
  Forecast_Method = "BM"  # please Mention the method name as BM or optical_flow
  os.makedirs('/content/tiff_files/vis/')
  os.mkdir('/content/tiff_files/aod/')
  os.mkdir('/content/tiff_files/pwd/')
  df_ghi_normal=pd.DataFrame(columns=['Date','GHI_'+AOI])
  df_ghi_forecast=pd.DataFrame(columns=['Date','Forecast_GHI_'+AOI])

## Cloud Detection Process

In [ ]:
# Loop for clear and cloudy pixel calculations
for subdir1 in os.listdir(dir):
  if subdir1=="Data": ## please change the folder name for calculatingg the clear n cloudy pixels
    subdir1=os.path.join(dir,subdir1) 
    for filename in os.listdir(subdir1):
      file=os.path.join(subdir1,filename)
      if(filename.endswith(".h5")):
        vis_clipped=initial_processing_cloud_mask(file,filename,dir,AOI)
        filter_mask_clipped=cloud_mask(file,dir,dir+'/content/vis_img_'+AOI+'/'+filename.split('_')[1]+'_'+filename.split('_')[2]+"_vis.png",AOI)
        filter_mask_clipped_clear=(filter_mask_clipped==0)
        filter_mask_clipped_cloudy=(filter_mask_clipped==1)
        vis_clipped_np=np.array(vis_clipped)
        clear_pixels=vis_clipped_np[filter_mask_clipped_clear]
        if clear_pixels.size!=0:
          for x in clear_pixels:
            df_clear.loc[-1]=x
            df_clear.index=df_clear.index+1
        cloudy_pixels=vis_clipped_np[filter_mask_clipped_cloudy]
        if cloudy_pixels.size!=0:
          for x in cloudy_pixels:
            df_cloudy.loc[-1]=x
            df_cloudy.index=df_cloudy.index+1
  df_clear.sort_index().to_csv(dir+'/clear and cloudy pixels/'+AOI+'_clear.csv')
  df_cloudy.sort_index().to_csv(dir+'/clear and cloudy pixels/'+AOI+'_cloudy.csv')

## GHI Calculation Process

In [ ]:
# Loop for GHI calculations
for subdir1 in os.listdir(dir):
    if subdir1=="Test_Data": ## please change the folder name for calculating the ghi values
      subdir1=os.path.join(dir,subdir1) 
      for subdir2 in os.listdir(subdir1): ## create separate folders for each day from the given data
        subdir2=os.path.join(subdir1,subdir2)
        filepaths  = sorted([os.path.join(subdir2, filename) for filename in os.listdir(subdir2)])
        for x in range(0,(len(filepaths)-2)):
          f1=h5py.File(filepaths[x],'r')
          filename1=filepaths[x].split('/')[8]
          vis1=initial_processing(filepaths[x],filename1,dir,AOI)
          vis_clipped1=clip_AOI(vis1,dir,AOI,Forecast_Method)
          clear=np.genfromtxt(dir+'/clear and cloudy pixels/'+AOI+'_clear.csv',dtype=int,delimiter=",",skip_header=1,usecols=1)
          cloudy=np.genfromtxt(dir+'/clear and cloudy pixels/'+AOI+'_cloudy.csv',dtype=int,delimiter=",",skip_header=1,usecols=1)
          minimum=np.percentile(clear,5)
          maximum=np.percentile(cloudy,98) 
          ci1=(vis_clipped1-minimum)/(maximum-minimum)
          ci_1=ci1.mean()
          if ci_1<0:
            ci_1=0
          elif ci_1>1:
            ci_1=1
          df_ghi_normal=GHI_Calculation(filepaths[x],filename1,dir,AOI,df_ghi_normal,ci_1)
          f2=h5py.File(filepaths[x+1],'r')
          filename2=filepaths[x+1].split('/')[8]
          vis2=initial_processing(filepaths[x+1],filename2,dir,AOI)
          vis_clipped2=clip_AOI(vis2,dir,AOI,Forecast_Method)
          clear=np.genfromtxt(dir+'/clear and cloudy pixels/'+AOI+'_clear.csv',dtype=int,delimiter=",",skip_header=1,usecols=1)
          cloudy=np.genfromtxt(dir+'/clear and cloudy pixels/'+AOI+'_cloudy.csv',dtype=int,delimiter=",",skip_header=1,usecols=1)
          minimum=np.percentile(clear,5)
          maximum=np.percentile(cloudy,98)
          ci2=(vis_clipped2-minimum)/(maximum-minimum)
          ci_2=ci2.mean()
          if ci_2<0:
            ci_2=0
          elif ci_2>1:
            ci_2=1
          df_ghi_normal=GHI_Calculation(filepaths[x+1],filename2,dir,AOI,df_ghi_normal,ci_2)
          filename3=filepaths[x+2].split('/')[8]
          c_f1=np.uint8(vis_clipped1)
          c_f2=np.uint8(vis_clipped2)
          if Forecast_Method=='BM':
            vis_forecast=BM(c_f2,c_f1) 
            if(vis_forecast is np.ndarray):
              sample_file=rio.open(dir+"/3DIMG_01JUL2021_0400_L1C_ASIA_MER_IMG_VIS.tif")
              desired_band=[0]
              with rio.open("/content/tiff_files/vis/"+filename3.split('_')[1]+'_'+filename3.split('_')[2]+"_vis.tiff","w",driver="Gtiff",height=sample_file.height,width=sample_file.width,count=1,dtype=vis_forecast.dtype,crs=sample_file.crs,transform=sample_file.transform) as dst:
                dst.write(vis_forecast,indexes=1)
              vis_t=rxr.open_rasterio("/content/tiff_files/vis/"+filename3.split('_')[1]+'_'+filename3.split('_')[2]+"_vis.tiff",variable=desired_band).squeeze()
              vis_forecast_clipped=vis_t
              clear=np.genfromtxt(dir+'/clear and cloudy pixels/'+AOI+'_clear.csv',dtype=int,delimiter=",",skip_header=1,usecols=1)
              cloudy=np.genfromtxt(dir+'/clear and cloudy pixels/'+AOI+'_cloudy.csv',dtype=int,delimiter=",",skip_header=1,usecols=1)
              minimum=np.percentile(clear,5)
              maximum=np.percentile(cloudy,98) 
              ci_forecast=(vis_forecast_clipped-minimum)/(maximum-minimum)
              ci_forecast_mean=ci_forecast.mean()
          elif Forecast_Method=='optical_flow':
            ci_forecast_mean=optical_flow_Method(ci1,ci2)
          if ci_forecast_mean<0:
            ci_forecast_mean=0
          elif ci_forecast_mean>1:
            ci_forecast_mean=1
          df_ghi_forecast=GHI_Calculation(filepaths[x+2],filename3,dir,AOI,df_ghi_forecast,ci_forecast_mean)

    df_ghi_normal.drop_duplicates(subset = ["Date"]).sort_values('Date').to_csv(dir+'/'+Forecast_Method+'_ghi_normal_'+AOI+'.csv')
    df_ghi_forecast.drop_duplicates(subset = ["Date"]).sort_values('Date').to_csv(dir+'/'+Forecast_Method+'_ghi_forecast_'+AOI+'.csv')


# Error metrics

In [ ]:
Normal_GHI_calculated=pd.read_csv(dir+'/'+Forecast_Method+'_ghi_normal_'+AOI+'.csv')
Forecast_GHI_calculated=pd.read_csv(dir+'/'+Forecast_Method+'_ghi_forecast_'+AOI+'.csv')
GHI_ground=pd.read_csv(dir+'/'+AOI+'_ground_data.csv')
n_col1=Normal_GHI_calculated['GHI_'+AOI]
f_col1=Forecast_GHI_calculated['Forecast_GHI_'+AOI]
col2=GHI_ground['GHI_Ground_Data']
normal_ghi_cor=n_col1.corr(col2)
print(normal_ghi_cor)
forecast_ghi_cor=n_col1.corr(col2)
print(forecast_ghi_cor)
n_val1=n_col1.values
f_val1=f_col1.values
val2=col2.values
normal_ghi_rmse=math.sqrt(mean_squared_error(n_val1, val2))
print(normal_ghi_rmse)
forecast_ghi_rmse=math.sqrt(mean_squared_error(f_val1, val2))
print(forecast_ghi_rmse)